In [27]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import re
from tqdm.notebook import tqdm
import string

scraper = sntwitter.TwitterSearchScraper('CSGO min_replies:1 min_faves:10')

tweets = []

# Przeglądamy 50 najpopularniejszych tweetów dla danego hasztagu
for i, tweet in enumerate(tqdm(scraper.get_items(), total=501)):
    data = [
        tweet.date,
        tweet.id,
        tweet.rawContent,
        tweet.user.username,
        tweet.likeCount,
        tweet.retweetCount,
        tweet.lang
    ]
    #print(data[2])
    data[2] = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet.rawContent).split())
    data[2] = data[2].lower()
    if data[2] == ' ' or data[2] == '':
        continue
    else:
        tweets.append(data)  # Dodajemy treść tweetu do listy tweets
        if i > 500:  # Przerywamy pętlę, gdy pobraliśmy 50 tweetów
            break

a=pd.DataFrame(tweets, columns=['date', 'id', 'content', 'username','like_count','retweet_count', 'language'])
a.head()


  0%|          | 0/501 [00:00<?, ?it/s]

,date,id,content,username,like_count,retweet_count,language
0,2023-03-29 11:21:37+00:00,1641037892955914241,langguth szilvai,Hi1PmYjX1jAiJZY,13,13,in
1,2023-03-29 11:21:03+00:00,1641037748957220864,khaleda hatsuaki,i6w83RgcPeN2Iew,16,16,et
2,2023-03-29 11:20:50+00:00,1641037696683638785,naoe kaseyama pg,xboy44215910,14,14,und
3,2023-03-29 11:19:11+00:00,1641037280881283072,hikkel kolla,Hi1PmYjX1jAiJZY,16,16,et
4,2023-03-29 11:17:28+00:00,1641036849668931584,svirezhev quinault mg,xboy44215910,13,13,und


In [28]:
a['date'] = a['date'].dt.tz_localize(None)
a.to_excel(r'export_dataframe.xlsx', index=False)